<a href="https://colab.research.google.com/github/Nniikkoollaass/data-science-modul-8-HW/blob/main/DS_HW_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [19]:
# Функція для читання даних з CSV файлів
def read_data(folder_path):
    data = pd.DataFrame()
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            data = pd.concat([data, df], ignore_index=True)
    return data

In [20]:
# Функція для обчислення часових ознак
def calculate_time_features(data):
    # Заповнюємо відсутні значення середнім
    data = data.fillna(data.mean())

    features = {}
    for axis in ['X', 'Y', 'Z']:
        axis_data = [col for col in data.columns if col.endswith('_' + axis)]
        for stat in ['mean', 'std', 'min', 'max', 'median']:
            features[axis + '_' + stat] = data[axis_data].agg(stat)
    return pd.DataFrame(features, index=[0])

In [22]:
# Зчитуємо дані та обчислюємо часові ознаки
idle_data = read_data("data/idle")
running_data = read_data("data/running")
stairs_data = read_data("data/stairs")
walking_data = read_data("data/walking")

idle_data = calculate_time_features(idle_data)
running_data = calculate_time_features(running_data)
stairs_data = calculate_time_features(stairs_data)
walking_data = calculate_time_features(walking_data)

# Об'єднуємо дані з різних класів
all_data = pd.concat([idle_data, running_data, stairs_data, walking_data], ignore_index=True)

# Розмічаємо дані
all_data["label"] = 0  # Припустимо, що 0 - idle, 1 - running, 2 - stairs, 3 - walking
all_data.loc[len(idle_data):len(idle_data) + len(running_data), "label"] = 1
all_data.loc[len(idle_data) + len(running_data):len(idle_data) + len(running_data) + len(stairs_data), "label"] = 2
all_data.loc[len(idle_data) + len(running_data) + len(stairs_data):, "label"] = 3

# Розділяємо дані на навчальний та тестувальний набори
X_train, X_test, y_train, y_test = train_test_split(all_data.drop("label", axis=1), all_data["label"], test_size=0.2, random_state=42)

# Заповнюємо відсутні значення за допомогою SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Стандартизуємо дані після заповнення відсутніх значень
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Навчаємо SVM модель
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)

# Навчаємо модель випадкового лісу
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Діагностичні відомості для SVM
svm_predictions = svm_model.predict(X_test_scaled)
print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))

# Діагностичні відомості для Random Forest
rf_predictions = rf_model.predict(X_test_scaled)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_predictions))

   X_mean  X_std  X_min  X_max  X_median  Y_mean  Y_std  Y_min  Y_max  \
0     NaN    NaN    NaN    NaN       NaN     NaN    NaN    NaN    NaN   

   Y_median  Z_mean  Z_std  Z_min  Z_max  Z_median  
0       NaN     NaN    NaN    NaN    NaN       NaN  


ValueError: Found array with 0 feature(s) (shape=(3, 0)) while a minimum of 1 is required by StandardScaler.